In [1]:
!pip install ultralytics

In [22]:
import os
import json
import shutil
from pathlib import Path
from tqdm import tqdm
from PIL import Image

# Define paths
coco_json_path = "/home/dylan/Downloads/FLIR_ADAS_v2/video_rgb_test/coco.json"  # Path to the COCO JSON file
output_dir = "./FLIR/RGB/images/test"  # Target YOLO output directory
annotations_dir = "./FLIR/RGB/labels/test"  # YOLO labels directory
images_dir = "/home/dylan/Downloads/FLIR_ADAS_v2/video_rgb_test/"  # Source images directory (if different)

# Create directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(annotations_dir, exist_ok=True)


In [23]:
# Load the COCO JSON file
with open(coco_json_path, 'r') as f:
    coco_data = json.load(f)

# Extract information from COCO JSON
images = coco_data['images']
annotations = coco_data['annotations']
categories = coco_data['categories']

# Map category IDs to names and YOLO class indices
category_map = {category['id']: i for i, category in enumerate(categories)}


In [24]:
def convert_bbox_to_yolo_format(bbox, img_width, img_height):
    """
    Convert COCO bbox [x_min, y_min, width, height] to YOLO format [x_center, y_center, width, height]
    """
    x_min, y_min, box_width, box_height = bbox
    x_center = x_min + box_width / 2
    y_center = y_min + box_height / 2

    # Normalize to [0, 1] range
    return [
        x_center / img_width,
        y_center / img_height,
        box_width / img_width,
        box_height / img_height,
    ]

# Group annotations by image_id
image_annotations = {img['id']: [] for img in images}
for annotation in annotations:
    image_annotations[annotation['image_id']].append(annotation)


In [25]:
# Process each image and its annotations
for img in tqdm(images, desc="Processing images"):
    # Load image metadata
    image_id = img['id']
    file_name = img['file_name']
    width, height = img['width'], img['height']
    source_image_path = os.path.join(images_dir, file_name)
    target_image_path = os.path.join(output_dir, file_name)
    
    # Ensure the directory tree exists for the target path
    os.makedirs(os.path.dirname(target_image_path), exist_ok=True)

    # Copy the image to the output directory
    if not os.path.exists(target_image_path):
        shutil.copy(source_image_path, target_image_path)

    # Create YOLO annotation file
    yolo_annotations = []
    for annotation in image_annotations[image_id]:
        category_id = annotation['category_id']
        yolo_class = category_map[category_id]
        bbox = annotation['bbox']
        yolo_bbox = convert_bbox_to_yolo_format(bbox, width, height)
        yolo_annotations.append([yolo_class, *yolo_bbox])

    # Ensure the directory tree exists for the annotation path
    yolo_label_path = os.path.join(annotations_dir, Path(file_name).stem + ".txt")
    os.makedirs(os.path.dirname(yolo_label_path), exist_ok=True)
    
    # Save YOLO annotations
    with open(yolo_label_path, 'w') as f:
        for yolo_annotation in yolo_annotations:
            f.write(" ".join(map(str, yolo_annotation)) + "\n")


Processing images: 100%|██████████████████| 3749/3749 [00:00<00:00, 3926.76it/s]


In [29]:
print("Hello World!")

Hello World!


In [36]:
import os

images_dir = "/home/dylan/projects/YOLO-RGB-T/notebooks/FLIR/RGB/images/train"
labels_dir = "/home/dylan/projects/YOLO-RGB-T/notebooks/FLIR/RGB/labels/train"

images = set(os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png')))
labels = set(os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt'))

missing_labels = images - labels
missing_images = labels - images

print("Images without labels:", missing_labels)
print("Labels without images:", missing_images)


Images without labels: set()
Labels without images: {'video-QTT9nPmNSQceyBezK-frame-002598-abPcWJNtssbDRykqT', 'video-GiDQGbWeWwtNTQEnG-frame-000618-f2ESyX6KKhcvwJkJa', 'video-XnyceSsWCsR7niZps-frame-007620-rZjxJdjpwypfioqYp', 'video-rXCGRrzyh98JMJk5v-frame-003572-JtQe7GzXKwE4YDXGz', 'video-JbQjHwPbbMvLcuoYg-frame-003352-XTng7uxpuo3uCkxXk', 'video-ptGfke8fvwJWvE8nW-frame-000951-xYquGF4KbEH7cz8Ys', 'video-j6zckHRzAN4XrXmTS-frame-000420-6K72vqkMhG5RrioC7', 'video-YneKJyFgKnCxo3Ypr-frame-005296-Njg4vnRyKwmYoE7DN', 'video-zN9k7HWQrEQaRWkWq-frame-005081-AYEYBA5kni95wkWbR', 'video-GiDQGbWeWwtNTQEnG-frame-000206-qDGz6ktg4gjkFzBcN', 'video-LxeHnzm8Rmb5j6DQ2-frame-001391-HjEzxnbHB9jiJ4NHA', 'video-HwAXFNSX2c48CdHch-frame-000600-WtBn2HFTzaovEWzuq', 'video-4ctFQTAqtiq8bzmHM-frame-002929-LKoCizGpXsbWHawpa', 'video-H4LTfF4eGmFxZL6ZS-frame-009619-vkKjh24C9L9kRZepy', 'video-hXJxXmduG5Cjz9FBg-frame-000564-GWRKLhKAQJXCwpLz4', 'video-cRcYAMzgJpmor4XfB-frame-009385-3dRitmA6FdG38Y6tP', 'video-WPJrL5MznEAg